<p>The following questions involve writing out equations and sketching out wavefunctions and the potential energy surface. Note, the sketches do not need to quantitatively accurate, they only need to show qualitative relationships.</p> 

<ol>
    <li>Write out the following equations with respect to the Harmonic Oscillator System:</li>
        <ol>
            <li>Kinetic Energy Operator</li>
            <li>Potential Energy Operator</li>
            <li>Hamiltonian</li>
            <li>Schrödinger Equation</li>
            <li>Wavefunction Energies</li>
        </ol>
    <li>Sketch a predicted potential energy surface for the harmonic oscillator.</li>
    <li>Sketch out the wavefunctions for the harmonic oscillator with v = 0, 1, 2 and 3.</li>
    <li>Sketch out the probability distributions for the harmonic oscillators with v = 0, 1, 2, and 3.</li>

### Graph Instructions

Click and drag your mouse within the graph to zoom into a specific portion of the graph. 
Use the plus/minus buttons at the top right corner of the graph in order to zoom in/out respectively. 
Click the house button in the upper right hand corner in order to reset the zoom to its default value. 

To hide a graph object, click on the object name in the right hand side of the graph. Click again to unhide the object. 
Double click on an object in the right hand side of the graph to have all the other objects become hidden except for that one object. Double click on the object again to have all the objects become visible again.

Below the graph there are several options that can be modified in order to interact with the graph. The options on the last row of options are manual, meaning that once they are updated the blue "Update Solution" must be clicked in order for the changes specified to be reflected on the graph. The options above the last row are dynamic options meaning that as soon as they are modified, the graph will automatically update to reflect the changes. Below follows a description of each of the options: 

<li>Resolution: Specifies how smooth the graph should be drawn, the higher the resolution the better the graph will look, but the slower it will respond </li>
<li>Precision: How many decimal places should be displayed when hovering over a point in the graph </li>
<li>Visible $\Psi$s: Specifies what wavefunctions should be visible on the graph. Individual levels are specified by their n value, and separate n values should be seprated by a semicolon Ex. 2;4;6;10, will only show the 2nd, 4th and 10th wavefunctions. Use a dash to only show the specified set of wavefunctions. Ex. 2-4, will show the 2nd, 3rd and 4th wavefunctions</li>
<li>Scale: Value specifies how to scale wavefunctions in order to better see their valleys and peaks</li>
<li>Start: Starting r position to draw graph at</li>
<li>End: Ending r position to stop drawing graph at</li>
<li>Mode: Allows for selection of being able to display wavefunctions, probability distributions, or both at the same time</li>
<li>Diatomic: Choose which diatomic to view</li>
<li>Comparison: Compare the Harmonic Oscillator against another Harmonic Oscillator or an Anharmonic Potential</li>
<li>$\mu$: When comparing against another Harmonic Osciallator, change the 2nd potential's reduced mass</li>
<li>$\omega$: When comparing against another Harmonic Osciallator, change the 2nd potential's 1st term vibrational constant</li>

In [1]:
#Allow Notebook to Import from Comp_Chem_Package
import sys
if("win" in sys.platform):
    sys.path.append("..\\..\\Backend")
else: 
    sys.path.append("../../Backend")
    
from nistScraper import getDiatomicConstants 
from morsePotential import morsePotential
from howPotential import howPotential
from wavefunction import wavefunction
from operators import HOperator
from schrodinger import schrod
from basisSet import basisSet
from compChemGlobal import *
from how import how
from rkr import rkr

from diatomicConstants import buildDiatomicConstants

scaleFactor = 900

diatomicSymbol = widgets.Dropdown(
    options = ["HCl", "DCl"],
    description = "Diatomic", 
    value = "HCl"
)
parameterOptions = ["Normal", "Times 2", "Divide 2"]
u = widgets.Dropdown(
    options = parameterOptions, 
    description = "μ", 
    value = "Normal"
)
w = widgets.Dropdown(
    options = parameterOptions, 
    description = "ω", 
    value = "Normal"
)
comparison = widgets.Dropdown(
    options = ["Harmonic", "Anharmonic"],
    description = "Comparison", 
    value = "Harmonic"
)
diatomic = widgets.HBox([widgets.Label("Diatomic Symbol"), diatomicSymbol, comparison, u, w])

calculate = startButton = widgets.ToggleButton(
    value = False, 
    description = "Update Solution",
    button_style = "info"
)

info = widgets.HTML("")
graph = widgets.VBox()
out = widgets.Output()
    
def inform(value):
    info.value = value
    out.clear_output()
    with out: 
        display(info)

#-------------------------------------------------------------------------------------

def buildSolution(dc, name = "", size = 25, color = None):
    pes =  howPotential(dc)
    pes.color = color
    basis = basisSet(dc, how, size)
    sol = schrod()
    sol.graphTitle = basis.diatomicConstants["name"] + " Harmonic Oscillator Schrödinger Solution"
    for index, bf in enumerate(basis):
        #eigen value analytical equation derived from hyperphysics website for harmonic osciallator
        
        ev = (index + 0.5) * dc["w"]
        evec = [1 if i == index else 0 for i in range(basis.size)]
        wf = wavefunction(evec, ev, basis, index).scale(scaleFactor)
        wf2 = wavefunction(evec, ev, basis, index, squared=True).scale(scaleFactor)
        wf.setGraphVariables(yEqualsCutoff = ev, color = color)
        wf2.setGraphVariables(yEqualsCutoff = [ev], color = color)
        
        sol.addGraphableObject(wf)
        sol.addGraphableObject(wf2)
        
    sol.addGraphableObject(pes)
    sol.scaleFactor = scaleFactor
    sol.pesLocations.append(len(sol.graphableObjects) - 1)
    
    return sol
 
#-------------------------------------------------------------------------------------
    
def howPESUpdate():
    dc = getDiatomicConstants(diatomicSymbol.value)
    size = 19 if diatomicSymbol.value == "HCl" else 23
    if(dc == False):
        info.value = "'" + diatomicSymbol.value + "' is not a valid diatomic molecule." 
        return
    info.value = "Calculating solution for " + diatomicSymbol.value + "..." 
    
    sol = buildSolution(dc, color = "red", size = size)
    name = ""
    #update diatomic constants
    if(comparison.value == "Harmonic"):
        for widget, index in [(u, "u"), (w, "w")]:
            if(widget.value == "Times 2"):
                dc[index] *= 2
                name += ", " + widget.description + " " + widget.value
            elif(widget.value == "Divide 2"):
                dc[index] /= 2
                name += ", " + widget.description + " " + widget.value
                
        if(name != ""):
            sol.combineSolutions(buildSolution(dc, name, size = size, color = "blue"), name)
    else: 
        sol2 = schrod()
        sol2.load(dc["name"], "Harmonic Oscillator", 120, "Extended Rydberg Potential", "RKR", scaleFactor = scaleFactor, color = "blue")
        sol.combineSolutions(sol2, " Anharmonic")
    
    endPoints = [.45, 3] if diatomicSymbol.value == "HCl" else [0.3, 2.65]
    sol.start, sol.end = endPoints[0], endPoints[1]
    graph.children = ( (sol.graph(getGraph = True), diatomic, calculate, info) )
    graph.children[0].children[0].update_xaxes(range=endPoints).update_yaxes(range=[0,sol.graphableObjects[1].energy + 5000])
    
    out.clear_output()
    with out: 
        display(graph)
    
    info.value = "Calculation Completed"

updateWrapper = lambda change :  howPESUpdate()

calculate.observe(updateWrapper, names="value")

display(out)
inform("Building Graph...")
howPESUpdate()  

Output()

<br></br>
<p>Answer the below qustions using the interactive graph of the v = 0, 1 and 2 wavefunctions.</p>

<ol start=5>
    <li>Compare your answers from the first set of questions to the plot. How did your predictions compare?</li>
    <li>Sketch and predict the potential energy surface for the harmonic oscillator for both HCl and DCl.</li>
    <li>Sketch the wavefunctions for the first 5 vibrational states for both HCl and DCl molecules.</li>
    <li>Describe the relationship between the reduced mass of each of the molecules and the width of the harmonic oscillator. Hint: you can also change the value for reduced mass in the plot to see differences in the width more clearly.</li>
    <li>Describe the relationship between the fundamental vibrational constant of each of the molecules and the width of the harmonic oscillator. Hint: you can also change the value for the fundamental vibrational frequency in the plot to see differences more clearly.</li>
    <li>Predict how many nodes the v = 5, 10, 500 and 1000 wavefunctions will have. What is the equation in terms of v you can use to predict the number of nodes?</li>

<br><br>
<p>Change the mode option in the graph above to "Probability Distribution"</p>

<ol start=11>
    <li>For the n = 0, 1, and 2 states answer the following questions for both HCl and DCl in terms of angstroms:</li>
    <ol>
        <li>What is the most probably location of the molecule?</li>
        <li>What is the average location of the molecule?</li>
        <li>Where will the molecule never be found? Give the specific name of this location.</li>
    </ol>
    </ol>

<br></br>
<p>The following questions will revolve around comparing a model of a diatomic molecule using a harmonic potential energy surface, and an anharmonic poetential energy surface. Specifically, the harmonic potential will be the harmonic oscillator presented above, while the anharmonic potential will be the anharmonic morse potential. On the graph above, change the Comparison mode from "Harmonic" to "Anharmonic". You may choose either HCl or DCl to compare the harmonic and anharmonic oscillators in the following questions.</p>

<ol start=12>
    <li>Which of the two molecules are you chosing to investigate?</li>
    <li>Why can a molecule represented by a harmonic oscillator not dissociate?</li> 
    <li>How will the energy levels of the harmonic and anharmonic potentials differ from one another?</li>
    <li>Which wavefunctions, if any, will be nearly identical between the harmonic and anharmonic potentials?</li>
    <li>Which potential (harmonic or anharmonic) best represents the actual behavior of diatomic molecules and why?</li>
</ol>

<br><br>
Survey Questions<br>
On a scale of 1 - 10, with 1 = I didn't enojoy this at all :( to 10 = I really enjoyed this!! :), how would you rate your interaction with the graph?<br>


<br>For this lab, would you rather interact with the prebuilt graph, or write your own code to answer the questions?<br>

<br>Did you experiance any technical issues with the graph?<br>

<br>Did the graph help in your understanding of the concepts and ideas being discussed in this lab?<br>

<br>Would you like to interact with this graph, and other similar quantum mechanical based interactive experiances outside of this lab and without any associated coursework? (Ex. Play around with it just for fun)?<br>

<br>Do you have any suggestions for improving the visuals/controls of the graph?</br>

<br>Were there any speed issues? (Graph very slow at updating, overall notebook felt sluggish?)</br>

<br>Do you feel your entire chemistry experiance can be improved through the use of more simulations such as the one presented here?</br>